#### Creacion de archivos para carga incremental tablas users,bussines, review

Preparacion para prueba user y bussines, se utilizara el archivo 'XXXXXX.json' para sacar tres conjuntos de dataset y se crearan 2 mas:<br>
    1. 'XXXXXX_inicial.json', se selecciona un 70% del dataset de forma aleatoria    <br>
    2. 'XXXXXX_incremental.json', se selecciona un 20% del dataset de forma aleatoria <br>
    3. 'XXXXXX_incremental_con_repeticiones.json' se selecciona 10% del dataset 
        pero se agregan valores de XXXXXX_incremental.json y XXXXXX_inicial.json <br>
    4. 'XXXXXX_incremental_con_fallas_columnas.json', Archivo .json con columnas aleatorio<br>
    5. 'XXXXXX_incremental_con fallas_formato_datos.json', Archivo .json con columnas que coinciden 
        con 'XXXXXX_inicial.json' pero su contenido no.

Preparacion para prueba user y bussines:<br>
    1. Se utilizan los id de las tablas user y bussines para crear la tabla 'review_inicial.json'
    2. De los valores que no quedaron en 'review_inicial.json' se crea con la mitad de ellos 'review_incremental.json'
    3. De los valores restantes se construye review_incremental_con_repeticiones.json' agregando valores
    que se encuentren tambien en 'review_incremental.json' y 'review_inicial.json'
    4. 'review_incremental_con_fallas_columnas.json', Archivo .json con columnas aleatorio<br>
    5. 'review_incremental_con fallas_formato_datos.json', Archivo .json con columnas que coinciden 
        con 'review_inicial.json' pero su contenido no.

### Condiciones de incio

In [4]:
#Librerias a utilizar
import numpy as np
import pandas as pd

In [3]:
#Direccion incial donde estan archivos de entrada y donde se almacenaran archivos salida
path_incial = 'C:\\Github\\DataSet_YELP\\'
path_final = 'C:\\Github\\DataSet_YELP\\pruebas_incremental\\'

### Funcion principal para creacion de datos de prueba user y bussines

In [3]:
def creacion_archivos_prueba(df,nombre,path_final):
    #Arbitrariamente se define:  
    #   70% de data en 'XXXXX_inicial.json' , 
    #   20% en 'XXXXX_incremental.json' y 
    #   10% en 'XXXXX_incremental_con_repeticiones.json'

    corte_0 = int(round(df.shape[0]*0.7,0))
    XXXXX_inicial = df.iloc[:corte_0,:]
    XXXXX_inicial.to_json(path_final+nombre+'_inicial.json')
    print('Creado :'+path_final+nombre+'_inicial.json')

    corte_1 = int(round(df.shape[0]*0.9,0))
    XXXXX_incremental = df.iloc[corte_0+1:corte_1,:]
    XXXXX_incremental.to_json(path_final+nombre+'_incremental.json')
    print('Creado :'+path_final+nombre+'_incremental.json')

    df1 = df.iloc[corte_1+1:,:]
    df2 = XXXXX_incremental.sample(df1.shape[0]//4)
    df3 = XXXXX_inicial.sample(df1.shape[0]//4)
    XXXXX_repeticiones = pd.concat([df1, df2,df3], ignore_index=True)
    XXXXX_repeticiones.to_json(path_final+nombre+'_incremental_con_repeticiones.json')
    print('Creado :'+path_final+nombre+'_incremental_con_repeticiones.json')

    df = pd.DataFrame(np.random.randn(1000, 4), columns=list('ABCD'))
    df.to_json(path_final+nombre+'_aleatorio1.json')
    print('Creado :'+path_final+nombre+'_aleatorio1.json')
    
    columnas = list(df.columns)
    df = pd.DataFrame(np.random.randn(1000, len(columnas)), columns=columnas)
    df.to_json(path_final+nombre+'_aleatorio2.json')
    print('Creado :'+path_final+nombre+'_aleatorio2.json')

### Creacion de datos de prueba user

In [5]:
nombre_archivo = 'user.json'
nombre = nombre_archivo.split('.')[0]

cont = 0
Tamano = 200000
for chunk in pd.read_json(path_incial+nombre_archivo,chunksize=Tamano, lines=True):
    cont = cont + 1
    #Se carga cada parte
    if cont == 1:  
        user = chunk
    else:
        user = pd.concat([user, chunk], ignore_index=True)

creacion_archivos_prueba(user,nombre,path_final)
#tiempo aprox = 4 min

Creado :C:\Github\DataSet_YELP\pruebas_incremental\user_inicial.json
Creado :C:\Github\DataSet_YELP\pruebas_incremental\user_incremental.json
Creado :C:\Github\DataSet_YELP\pruebas_incremental\user_incremental_con_repeticiones.json
Creado :C:\Github\DataSet_YELP\pruebas_incremental\user_aleatorio1.json
Creado :C:\Github\DataSet_YELP\pruebas_incremental\user_aleatorio2.json


### Creacion de datos de prueba bussines

In [24]:
nombre_archivo = 'business.json'
nombre = nombre_archivo.split('.')[0]
business = pd.read_json(path_incial+nombre_archivo,lines=True)
creacion_archivos_prueba(business,nombre,path_final)

Creado :C:\Github\DataSet_YELP\pruebas_incremental\business_inicial.json
Creado :C:\Github\DataSet_YELP\pruebas_incremental\business_incremental.json
Creado :C:\Github\DataSet_YELP\pruebas_incremental\business_incremental_con_repeticiones.json
Creado :C:\Github\DataSet_YELP\pruebas_incremental\business_aleatorio1.json
Creado :C:\Github\DataSet_YELP\pruebas_incremental\business_aleatorio2.json


### Creacion de datos de prueba review

In [2]:
nombre_archivo = 'review.json'
nombre = nombre_archivo.split('.')[0]

In [5]:
nombre_archivo = 'review.json'
nombre = nombre_archivo.split('.')[0]
cont = 0
Tamano = 200000
for chunk in pd.read_json(path_incial+nombre_archivo,chunksize=Tamano, lines=True):
    cont = cont + 1
    #Se carga cada parte
    if cont == 1:
        review = chunk
    else:
        review = pd.concat([review, chunk], ignore_index=True)
#tiempo aprox = 10 min

In [6]:
user_inicial = pd.read_json(path_final+'user_inicial.json')
ids_user_inicial = user_inicial['user_id']
#tiempo aprox 3min

In [7]:
business_inicial = pd.read_json(path_final+'business_inicial.json')
ids_business_inicial = business_inicial['business_id']

In [9]:
df = pd.merge(ids_business_inicial,review,left_on='business_id',right_on='business_id')
review_inicial = pd.merge(ids_user_inicial,df,left_on='user_id',right_on='user_id')
#Tiempo aprox = 25min 

In [12]:
review_inicial.to_json(path_final+nombre+'_inicial.json')
print('Creado :'+path_final+nombre+'_inicial.json')
#tiempo 7min

Creado :C:\Github\DataSet_YELP\pruebas_incremental\review_inicial.json


In [21]:
listado1 =list(review_inicial['review_id'])
review_incremental_completo = review[~review['review_id'].isin(listado1)]
review_incremental_completo.to_json(path_final+nombre+'_incremental_completo.json')
print('Creado :'+path_final+nombre+'_incremental_completo.json')

Creado :C:\Github\DataSet_YELP\pruebas_incremental\review_incremental_completo.json


In [32]:
#Arbitrariamente se define:  
#   70% en 'review_incremental.json' y 
#   30% en 'review_incremental_con_repeticiones.json'

review_incremental_completo = review_incremental_completo.reset_index(drop=True)
corte_0 = int(round(review_incremental_completo.shape[0]*0.7,0))
review_incremental = review_incremental_completo.iloc[0:corte_0,:]
review_incremental.to_json(path_final+nombre+'_incremental.json')
print('Creado :'+path_final+nombre+'_incremental.json')

df1 = review_incremental_completo.iloc[corte_0+1:,:]
df2 = review_incremental.sample(df1.shape[0]//4)
df3 = review_inicial.sample(df1.shape[0]//4)
user_incremental_con_repeticiones = pd.concat([df1, df2,df3], ignore_index=True)
user_incremental_con_repeticiones.to_json(path_final+nombre+'_incremental_con_repeticiones.json')
print('Creado :'+path_final+nombre+'_incremental_con_repeticiones.json')


Creado :C:\Github\DataSet_YELP\pruebas_incremental\review_incremental_con_repeticiones.json


In [33]:
df = pd.DataFrame(np.random.randn(1000, 4), columns=list('ABCD'))
df.to_json(path_final+nombre+'_aleatorio1.json')
print('Creado :'+path_final+nombre+'_aleatorio1.json')
    
columnas = list(review_inicial.columns)
df = pd.DataFrame(np.random.randn(1000, len(columnas)), columns=columnas)
df.to_json(path_final+nombre+'_aleatorio2.json')
print('Creado :'+path_final+nombre+'_aleatorio2.json')

Creado :C:\Github\DataSet_YELP\pruebas_incremental\review_aleatorio1.json
Creado :C:\Github\DataSet_YELP\pruebas_incremental\review_aleatorio2.json
